# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [1]:
import re
def preprocess_text(text: str) -> list:
    text = text.lower()
    return re.findall(r"\w+\b", text)

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = preprocess_text(reference_text)
    recognized_words = preprocess_text(recognized_text)
    # TODO

    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            m_i_j = 0 if reference_words[i-1] == recognized_words[j-1] else 1
            distance_matrix[i][j] = min(distance_matrix[i][j-1]+1, # вставка
                                       distance_matrix[i-1][j]+1, # удаление
                                       distance_matrix[i-1][j-1] + m_i_j) # замена
    lev_distance = distance_matrix[-1][-1]

    # Расчет WER  (в процентах)
    wer = lev_distance / len(reference_words) * 100
    return wer

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [2]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [3]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

In [4]:

def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = preprocess_text(reference_text)
    recognized_words = preprocess_text(recognized_text)
    
    # расстояние Левенштейна
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            m_i_j = 0 if reference_words[i-1] == recognized_words[j-1] else 1
            distance_matrix[i][j] = min(distance_matrix[i][j-1] + 1,  # вставка
                                        distance_matrix[i-1][j] + 1,  # удаление
                                        distance_matrix[i-1][j-1] + m_i_j)  # замена

    # Финальное расстояние Левенштейна
    lev_distance = distance_matrix[-1][-1]

    # Расчет WER  (в процентах)
    wer = lev_distance / len(reference_words) * 100

    # Выравнивание и подсчет операций
    correct, deletion, insertion, substitution = 0, 0, 0, 0
    ali_ref, ali_rec, ali_ann = [], [], []

    i, j = len(reference_words), len(recognized_words)
    
    while i > 0 or j > 0:
        ref_word = reference_words[i-1]
        rec_word = recognized_words[j-1]
        if ref_word == rec_word:  # Совпадение
            ali_ref.append(ref_word)
            ali_rec.append(rec_word)
            ali_ann.append("C")
            correct += 1
            i -= 1
            j -= 1
        elif distance_matrix[i][j] == distance_matrix[i-1][j-1] + 1: # Замена
            ali_ref.append(ref_word)
            ali_rec.append(rec_word)
            ali_ann.append("S")
            substitution += 1
            i -= 1
            j -= 1
        elif distance_matrix[i][j] == distance_matrix[i-1][j] + 1:  # Удаление
            ali_ref.append(ref_word)
            ali_rec.append("***")
            ali_ann.append("D")
            deletion += 1
            i -= 1
        else:  # Вставка
            ali_ref.append("***")
            ali_rec.append(rec_word)
            ali_ann.append("I")
            insertion += 1
            j -= 1

    # Так как матрица проходилась с конца (с последней строки последнего столбца)
    ali_ref.reverse()
    ali_rec.reverse()
    ali_ann.reverse()

    ali = [ali_ref, ali_rec, ali_ann]

    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"

    return {
        "wer": wer,
        "cor": correct,
        "del": deletion,
        "ins": insertion,
        "sub": substitution,
        "ali": ali
    }


In [5]:
# Закомментила, чтобы при запуске сразу всех ячеек не падали тесты
#def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
#    return {
#            "wer" : 0,
#            "cor": 2, 
#            "del": 0,
#            "ins": 0,
#            "sub": 0,
#            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]}

In [6]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно RichTranscriptErrorRate (RTER) по формуле

$$ RTER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [7]:
def preprocess_for_per(text: str) -> list:
    text = text.lower()
    return re.findall(r"\w+|[^\w\s]", text)

def calculate_wer_per(reference_text: str, recognized_text: str):
    wer = calculate_wer(reference_text, recognized_text)

    reference_words_punkts = preprocess_for_per(reference_text)
    recognized_words_punkts = preprocess_for_per(recognized_text)

    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words_punkts) + 1) for _ in range(len(reference_words_punkts) + 1)]

    # Наполняем первую строку и первый столбец матрицы
    for i in range(len(reference_words_punkts) + 1):
        distance_matrix[i][0] = i
    for j in range(len(recognized_words_punkts) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words_punkts) + 1):
        for j in range(1, len(recognized_words_punkts) + 1):
            ref_word = reference_words_punkts[i-1]
            rec_word = recognized_words_punkts[j-1]

            # Если это одинаковые слова или знаки препинания, то это совпадение
            if ref_word == rec_word:
                m_i_j = 0
            # Если одно из двух слов – знак препинания, то это НЕ замена
            elif (re.match(r'\w+', ref_word) and not re.match(r'\w+', rec_word)) or (not re.match(r'\w+', ref_word) and re.match(r'\w+', rec_word)):
                m_i_j = 0
            # Иначе это замена
            else:
                m_i_j = 1

            # Считаем минимальное расстояние
            distance_matrix[i][j] = min(
                distance_matrix[i][j-1] + 1,  # вставка
                distance_matrix[i-1][j] + 1,  # удаление
                distance_matrix[i-1][j-1] + m_i_j  # замена
            )

    # Выравнивание и подсчет операций
    correct, insertion, deletion, substitution = 0, 0, 0, 0
    correct_punctuation, insertion_punctuation, deletion_punctuation, substitution_punctuation = 0, 0, 0, 0
    ali_ref, ali_rec, ali_ann = [], [], []

    i, j = len(reference_words_punkts), len(recognized_words_punkts)

    while i > 0 or j > 0:
        ref_word = reference_words_punkts[i-1]
        rec_word = recognized_words_punkts[j-1]
        if ref_word == rec_word:  # Совпадение
            ali_ref.append(ref_word)
            ali_rec.append(rec_word)
            if re.match(r'[^\w\s]', ref_word):  # Совпадение по пунктуации
                correct_punctuation += 1
                ali_ann.append("C_p")
            else:
                correct += 1
                ali_ann.append("C")
            i -= 1
            j -= 1
        elif distance_matrix[i][j] == distance_matrix[i-1][j-1] + 1:  # Замена
            ali_ref.append(ref_word)
            ali_rec.append(rec_word)
            if re.match(r'[^\w\s]', ref_word) and re.match(r'[^\w\s]', rec_word):
                substitution_punctuation += 1
                ali_ann.append("S_p")
            else:
                substitution +=1
                ali_ann.append("S")
            i -= 1
            j -= 1
        elif distance_matrix[i][j] == distance_matrix[i-1][j] + 1:  # Удаление
            ali_ref.append(ref_word)
            ali_rec.append("***")            
            if re.match(r'[^\w\s]', ref_word):
                deletion_punctuation += 1
                ali_ann.append("D_p")
            else:
                deletion += 1
                ali_ann.append("D")
            i -= 1
        else:  # Вставка
            if re.match(r'[^\w\s]', rec_word):  # Вставка пунктуации
                ali_ref.append("***")
                ali_rec.append(rec_word)
                insertion_punctuation += 1
                ali_ann.append("I_p")
                j -= 1
            # Условие ниже – чтобы отлавливать случаи, как в тесте 3, когда по формуле вставка, но на самом деле удаление
            if re.match(r'[^\w\s]', ref_word) and not re.match(r'[^\w\s]', rec_word):
                ali_rec.append("***")
                ali_ref.append(ref_word)
                deletion_punctuation += 1
                ali_ann.append("D_p")
                i -= 1
            elif not re.match(r'[^\w\s]', rec_word):  # Обычная вставка
                ali_ref.append("***")
                ali_rec.append(rec_word)
                insertion += 1
                ali_ann.append("I")
                j -= 1

    ali_ref.reverse()
    ali_rec.reverse()
    ali_ann.reverse()

    ali = [ali_ref, ali_rec, ali_ann]

    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"

    # Подсчет PER
    numerator = insertion_punctuation + deletion_punctuation + substitution_punctuation
    denominator = correct_punctuation + substitution_punctuation + deletion_punctuation

    per = (numerator / denominator) * 100 if denominator > 0 else 0

    return {"wer" : wer,
            "per": per, 
            "ali": ali}


In [8]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [9]:
from itertools import permutations
# Тк в основной функции подсчёта SA-WER есть цикл, показалось, что лучше вынести выравнивание в отдельную функцию
def get_alignment_sawer(distance_matrix, reference_text, recognized_text, ref_speakers_to_id, recognized_speakers):
    # Выравнивание и подсчет операций
    correct, deletion, insertion, substitution, speaker_incorrect, speaker_correct = 0, 0, 0, 0, 0, 0
    ali_ref, ali_rec, ali_ann, ali_ref_speaker, ali_rec_speaker, ali_ann_speaker = [], [], [], [], [], []

    i, j = len(reference_text), len(recognized_text)
    
    while i > 0 or j > 0:
        ref_word = reference_text[i-1]
        rec_word = recognized_text[j-1]
        ref_speaker = ref_speakers_to_id[i-1]
        rec_speaker = recognized_speakers[j-1]
        if ref_speaker == rec_speaker:
            speaker_correct += 1
            ali_ref_speaker.append(ref_speaker)
            ali_rec_speaker.append(rec_speaker)
            ali_ann_speaker.append("S_C")
        else:
            speaker_incorrect += 1
            ali_ref_speaker.append(ref_speaker)
            ali_rec_speaker.append(rec_speaker)
            ali_ann_speaker.append("S_I")
            
        if ref_word == rec_word:  # Совпадение
            ali_ref.append(ref_word)
            ali_rec.append(rec_word)
            ali_ann.append("C")
            correct += 1
            i -= 1
            j -= 1
        elif distance_matrix[i][j] == distance_matrix[i-1][j-1] + 1: # Замена
            ali_ref.append(ref_word)
            ali_rec.append(rec_word)
            ali_ann.append("S")
            substitution +=1
            i -= 1
            j -= 1
        elif distance_matrix[i][j] == distance_matrix[i-1][j] + 1:  # Удаление
            ali_ref.append(ref_word)
            ali_rec.append("***")
            ali_ann.append("D")
            deletion += 1
            i -= 1
        else:  # Вставка
            ali_ref.append("***")
            ali_rec.append(rec_word)
            ali_ann.append("I")
            insertion += 1
            j -= 1

    ali_ref.reverse()
    ali_rec.reverse()
    ali_ann.reverse()
    ali_ref_speaker.reverse()
    ali_rec_speaker.reverse()
    ali_ann_speaker.reverse()

    for ali_ann_item, ali_ann_speaker_item in zip(ali_ann, ali_ann_speaker):
        # Если слово правильное, а диктор распознан неправильно, то это не считается совпадением
        if ali_ann_item == "C" and ali_ann_speaker_item != "S_C":
            correct -= 1
        # Если слово удалено, но есть ошибка диктора, то этот случай не входит в число ошибок диктора
        if ali_ann_item == "D" and ali_ann_speaker_item == "S_I":
            speaker_incorrect -= 1

    ali = [ali_ref, ali_rec, ali_ann, ali_ref_speaker, ali_rec_speaker, ali_ann_speaker]

    assert len(ali[0]) == len(ali[1]) == len(ali[2]) == len(ali[3]) == len(ali[4]) == len(ali[5]), f"wrong ali {ali}"

    numerator = insertion + deletion + substitution + speaker_incorrect
    denominator = deletion + substitution + correct + speaker_incorrect

    sawer = numerator / denominator * 100

    return sawer, ali


In [10]:
def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)
    
    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER
    
    # расстояние Левенштейна
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_text) + 1) for _ in range(len(reference_text) + 1)]
    
    # Наполнение первой строки матрицы
    for i in range(len(reference_text) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_text) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_text) + 1):
        for j in range(1, len(recognized_text) + 1):
            m_i_j = 0 if reference_text[i-1] == recognized_text[j-1] else 1
            distance_matrix[i][j] = min(distance_matrix[i][j-1] + 1,  # вставка
                                        distance_matrix[i-1][j] + 1,  # удаление
                                        distance_matrix[i-1][j-1] + m_i_j)  # замена

    unique_reference_speakers = list(set(reference_speakers))
    unique_recognized_speakers = list(set(recognized_speakers))
    # Строчки ниже нужны для подсчёта ошибок в том случае, когда распознанный диктор только один
    if len(unique_recognized_speakers) < 2:
        unique_recognized_speakers.extend(list(range(len(unique_reference_speakers) - 1)))

    sawers_alis = {}
    # Подсчёт значения ошибки для каждого варианта маппинга
    for perm in permutations(unique_recognized_speakers, len(unique_reference_speakers)):
        # Маппинг
        ref_speakers_to_id_mapping = {ref_sp: rec_sp for ref_sp, rec_sp in zip(unique_reference_speakers, perm)}
        ref_speakers_to_id = [ref_speakers_to_id_mapping[s] for s in reference_speakers]
        # Подсчёт ошибки, получение выравнивания реализованы в отдельной функции выше
        sawer, ali = get_alignment_sawer(distance_matrix, reference_text, recognized_text, ref_speakers_to_id, recognized_speakers)
        sawers_alis[sawer] = ali
    
    min_sawer = min(sawers_alis.keys())
    ali = sawers_alis[min_sawer]

    sawer=min_sawer
    ali=ali

    return {"sawer" : sawer, 
            "ali": ali}

In [11]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

Test 3 passed
